<a href="https://colab.research.google.com/github/DJCordhose/mlops-drift/blob/main/notebooks/drift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detecting drift

* you can let evidently choose the test or make your own choice
  * allows to select per type or per feature: https://docs.evidentlyai.com/reference/api-reference/evidently.options
  * which test: https://www.evidentlyai.com/blog/data-drift-detection-large-datasets
* Drift algorithm: https://docs.evidentlyai.com/reference/data-drift-algorithm  
* complete code in `src/insurance_prediction/monitoring/data_drift.py`
  * implemented as a decorator: https://docs.python.org/3/search.html?q=decorator
  * uses rolling window of incoming request in memory buffer
  

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
IN_COLAB

True

In [2]:
def url_for_dataset(relative_path):
  if IN_COLAB:
    url = f'https://github.com/DJCordhose/mlops-drift/raw/main/datasets/insurance_prediction/{relative_path}.csv.gz'
  else:
    url =  f'../datasets/insurance_prediction/{relative_path}.csv.gz'
  return url

In [3]:
if IN_COLAB:
  !pip install -q evidently

In [4]:
# how many months after training?
# iteration = 36 # final month in dataset, should have the biggest difference
iteration = 12 # one year later, drift should starting to show

In [5]:
reference_dataset_url = url_for_dataset(f'reference')
current_dataset_url = url_for_dataset(f'monthly/month-{iteration}')

In [6]:
from evidently.metrics import DatasetDriftMetric
from evidently.base_metric import InputData, ColumnMapping
from evidently.runner.loader import DataLoader, DataOptions
from evidently.calculations.data_drift import get_drift_for_columns
from evidently.options import DataDriftOptions
from evidently.utils.data_operations import process_columns

In [7]:
# could just as well be pandas code
reference_dataset = DataLoader().load(
   filename=reference_dataset_url,
   data_options = DataOptions(date_column=None, separator=';'),
)
reference_dataset = reference_dataset.drop(['risk', 'group', 'group_name'], axis='columns')
reference_dataset.head()

,training,age,emergency_braking,braking_distance,power,miles
0,0,41.951100,0,40.933328,122.238329,98.509765
1,0,24.368286,1,44.797317,113.765298,46.324178
2,0,18.314649,1,41.587241,143.427269,76.862968
3,0,51.265254,1,47.266716,111.578133,102.697069
4,0,23.578861,0,42.835319,145.994235,63.690055


In [8]:
# could just as well be pandas code
current_dataset = DataLoader().load(
   filename=current_dataset_url,
   data_options = DataOptions(date_column=None, separator=';'),
)
current_dataset = current_dataset.drop(['risk', 'group', 'group_name'], axis='columns')
current_dataset.head()

,training,age,emergency_braking,braking_distance,power,miles
0,0,18.936814,0,43.758711,96.338095,63.786116
1,0,32.041673,1,29.122178,187.562915,81.079895
2,0,27.554665,1,40.783638,123.967462,49.117011
3,0,30.040739,1,37.612380,167.705733,67.576213
4,0,20.513682,1,37.577958,137.666668,61.367460


In [9]:
column_mapping = ColumnMapping(
    categorical_features=['training', 'emergency_braking'],
    numerical_features=['age', 'braking_distance', 'power', 'miles'],
    prediction=None,
    target=None,
)
columns = column_mapping.categorical_features + column_mapping.numerical_features
drift_share = 0.5

In [10]:
input_data = InputData(
    reference_data=reference_dataset,
    current_data=current_dataset,
    column_mapping=column_mapping,
    current_additional_features=None,
    data_definition=None,
    reference_additional_features=None
)

In [11]:
dataset_columns = process_columns(input_data.reference_data, input_data.column_mapping)
result = get_drift_for_columns(
    current_data=input_data.current_data,
    reference_data=input_data.reference_data,
    data_drift_options=DataDriftOptions(),
    drift_share_threshold=drift_share,
    dataset_columns=dataset_columns,
    columns=column_mapping.categorical_features + column_mapping.numerical_features,
    agg_data = True
)

In [12]:
result.number_of_columns

6

In [13]:
result.number_of_drifted_columns

2

In [14]:
result.share_of_drifted_columns

0.3333333333333333

In [15]:
result.dataset_drift

False

In [16]:
for column in columns:
    drift = result.drift_by_columns[column]
    print(f'{column}: {drift.drift_score} ({drift.stattest_name}){", drifted" if drift.drift_detected else ""}')

training: 0.005181253622581234 (Jensen-Shannon distance)
emergency_braking: 0.13147841305705196 (Jensen-Shannon distance), drifted
age: 0.09701144919398294 (Wasserstein distance (normed))
braking_distance: 0.0692912211638476 (Wasserstein distance (normed))
power: 0.027957726384958408 (Wasserstein distance (normed))
miles: 0.22422961324847054 (Wasserstein distance (normed)), drifted
